# 🍏 건강 리소스 검색 에이전트 튜토리얼 🍎

**건강 리소스 검색 에이전트** 자습서에 오신 것을 환영합니다! **Azure AI Foundry** SDK를 사용하여 어시스턴트를 빌드하겠습니다:

1. 건강 및 레시피 파일을 벡터 스토어에 **업로드**합니다.
2. **파일 검색** 도구를 사용하여 **에이전트를 생성**합니다.
3. 관련 식단 정보를 **검색**합니다.
4. 건강 및 웰빙 관련 질문에 **답변**합니다(면책 조항 포함!).

### ⚠️ 중요 의료 고지 사항 ⚠️
> **이 노트의 모든 건강 정보는 일반적인 교육 목적으로만 제공되며 전문적인 의학적 조언, 진단 또는 치료를 대신할 수 없습니다.** 궁금한 점이 있으면 항상 자격을 갖춘 의료 전문가의 조언을 구하세요.

## 전제 조건
- Agent 기본 노트북 전체 - [1-basics.ipynb](1-basics.ipynb)
- **역할**  
  1. Azure AI 파운드리 프로젝트의 **Azure AI Developer** 
  2. 프로젝트의 저장소 계정의 **Storage Blob Data Contributor** 
  3. 자체 검색 리소스와 함께 표준 에이전트 설정을 사용하는 경우, 해당 리소스에  **Cognitive Search Data Contributor**도 있는지 확인합니다.


## 검색을 시작해 봅시다!
몇 가지 샘플 파일을 업로드하고, 이를 위한 벡터 저장소를 만든 다음, 이러한 리소스에서 식단 지침, 레시피 등을 검색할 수 있는 에이전트를 가동하는 방법을 보여드리겠습니다. 즐겨보세요!

<img src="./seq-diagrams/3-file-search.png" width="30%"/>


## 1. 초기 설정
여기서는 필요한 라이브러리를 가져오고, `.env`에서 환경 변수를 로드하고, **AIProjectClient**를 초기화합니다. 해봅시다! 🎉

In [ ]:
import os
import time
from pathlib import Path

from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    FileSearchTool,
    FilePurpose,
    MessageTextContent,
    MessageRole
)

# Load environment variables from parent .env
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Initialize AIProjectClient
try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=os.environ.get("PROJECT_CONNECTION_STRING")
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"❌ Error initializing project client: {e}")

## 2. 샘플 파일 준비하기 🍲🗒
레시피와 가이드라인을 위한 더미 .md 파일을 만들겠습니다. 그런 다음 검색을 위해 벡터 스토어에 저장합니다.


In [ ]:
def create_sample_files():
    recipes_md = (
        """# Healthy Recipes Database\n\n"
        "## Gluten-Free Recipes\n"
        "1. Quinoa Bowl\n"
        "   - Ingredients: quinoa, vegetables, olive oil\n"
        "   - Instructions: Cook quinoa, add vegetables\n\n"
        "2. Rice Pasta with Vegetables\n"
        "   - Ingredients: rice pasta, mixed vegetables\n"
        "   - Instructions: Boil pasta, sauté vegetables\n\n"
        "## Diabetic-Friendly Recipes\n"
        "1. Low-Carb Stir Fry\n"
        "   - Ingredients: chicken, vegetables, tamari sauce\n"
        "   - Instructions: Cook chicken, add vegetables\n\n"
        "2. Greek Salad\n"
        "   - Ingredients: cucumber, tomatoes, feta, olives\n"
        "   - Instructions: Chop vegetables, combine\n\n"
        "## Heart-Healthy Recipes\n"
        "1. Baked Salmon\n"
        "   - Ingredients: salmon, lemon, herbs\n"
        "   - Instructions: Season salmon, bake\n\n"
        "2. Mediterranean Bowl\n"
        "   - Ingredients: chickpeas, vegetables, tahini\n"
        "   - Instructions: Combine ingredients\n"""
    )

    guidelines_md = (
        """# Dietary Guidelines\n\n"
        "## General Guidelines\n"
        "- Eat a variety of foods\n"
        "- Control portion sizes\n"
        "- Stay hydrated\n\n"
        "## Special Diets\n"
        "1. Gluten-Free Diet\n"
        "   - Avoid wheat, barley, rye\n"
        "   - Focus on naturally gluten-free foods\n\n"
        "2. Diabetic Diet\n"
        "   - Monitor carbohydrate intake\n"
        "   - Choose low glycemic foods\n\n"
        "3. Heart-Healthy Diet\n"
        "   - Limit saturated fats\n"
        "   - Choose lean proteins\n"""
    )

    # Save to local .md files
    with open("recipes.md", "w", encoding="utf-8") as f:
        f.write(recipes_md)
    with open("guidelines.md", "w", encoding="utf-8") as f:
        f.write(guidelines_md)

    print("📄 Created sample resource files: recipes.md, guidelines.md")
    return ["recipes.md", "guidelines.md"]

sample_files = create_sample_files()

#### ✨ 검색 권한에 대한 참고 사항
벡터 스토어를 만들 때 Azure AI 검색 리소스에 **Cognitive Search Data Contributor** 역할도 있어야 합니다(자체 검색 리소스와 함께 표준 에이전트 설정을 사용하는 경우). 이 역할이 없으면 **Forbidden** 오류가 발생하는 경우가 많습니다. 권한 구성에 대한 자세한 내용은 [Authentication Setup](../../1-introduction/1-authentication.ipynb#4-add-agent-service-permissions) 을 참조하세요.


## 3. 벡터 스토어 만들기 📚
새로 만든 파일을 업로드하고 검색을 위해 하나의 벡터 스토러로 그룹화합니다. 이렇게 하면 agent가 나중에 관련 텍스트를 찾을 수 있습니다.

In [ ]:
def create_vector_store(files, store_name="my_health_resources"):
    try:
        # Step 1: Upload files to Azure AI Agent service
        # Each file needs to be uploaded individually and we'll collect their IDs
        uploaded_ids = []
        for fp in files:
            # upload_file_and_poll ensures the upload is complete before continuing
            # FilePurpose.AGENTS tells the service these files are for agent usage
            upl = project_client.agents.upload_file_and_poll(
                file_path=fp,
                purpose=FilePurpose.AGENTS
            )
            uploaded_ids.append(upl.id)
            print(f"✅ Uploaded: {fp} -> File ID: {upl.id}")

        # Step 2: Create a vector store from the uploaded files
        # A vector store converts text into numerical vectors for semantic search
        # create_vector_store_and_poll waits until indexing is complete
        vs = project_client.agents.create_vector_store_and_poll(
            file_ids=uploaded_ids,  # Pass all our uploaded file IDs
            name=store_name         # Give our vector store a friendly name
        )
        print(f"🎉 Created vector store '{store_name}', ID: {vs.id}")
        return vs, uploaded_ids
    except Exception as e:
        print(f"❌ Error creating vector store: {e}")
        return None, []

# Initialize empty variables to store our vector store and file IDs
vector_store, file_ids = None, []

# If we successfully created sample files earlier, create a vector store from them
if sample_files:
    vector_store, file_ids = create_vector_store(sample_files, "health_resources_example")

## 4. 건강 리소스 에이전트 생성하기 🔎
새로 만든 벡터 스토어를 가리키는 **FileSearchTool**를 사용할 예정입니다. 다음 고지 사항, 식이요법 도움말 등에 대한 지침이 포함된 에이전트를 생성합니다.

In [ ]:
def create_health_resource_agent(vstore_id):
    try:
        # Create a FileSearchTool that will allow our agent to search through documents
        # FileSearchTool uses the vector store we created earlier to perform semantic search
        # This means the agent can find relevant content even if the exact words don't match
        file_search_tool = FileSearchTool(vector_store_ids=[vstore_id])

        # Create an AI agent that will use our search tool and follow specific instructions
        # The agent combines:
        # 1. A base LLM model (specified in environment variables or defaults to gpt-4o-mini)
        # 2. The ability to search our health documents using the FileSearchTool
        # 3. Custom instructions that shape how it responds to questions
        agent = project_client.agents.create_agent(
            # Specify which LLM model to use - fallback to gpt-4o-mini if not set
            model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini"),
            
            # Give our agent a descriptive name
            name="health-search-agent",
            
            # These instructions act like a personality and rule set for our agent
            # They ensure consistent, responsible health advice
            instructions="""
                You are a health resource advisor with access to dietary and recipe files.
                You:
                1. Always present disclaimers (you're not a doctor!)
                2. Provide references to the files when possible
                3. Focus on general nutrition or recipe tips.
                4. Encourage professional consultation for more detailed advice.
            """,
            
            # Connect the search tool's interface definition and required resources
            # This gives the agent the ability to actually perform searches
            tools=file_search_tool.definitions,
            tool_resources=file_search_tool.resources
        )
        print(f"🎉 Created health resource agent, ID: {agent.id}")
        return agent
    except Exception as e:
        print(f"❌ Error creating health resource agent: {e}")
        return None

# Initialize our agent variable
health_agent = None

# Only create the agent if we successfully created a vector store earlier
if vector_store:
    health_agent = create_health_resource_agent(vector_store.id)

## 5. 건강 관련 자료 검색 🏋️👩‍🍳
새 대화 스레드를 만들고 “Gluten-free recipe ideas?” 또는 “Heart-healthy meal plan?”과 같은 질문을 합니다.에이전트가 벡터 스토어에서 파일 검색을 수행하여 관련 정보를 찾습니다.

In [ ]:
def create_search_thread(agent):
    try:
        # In Azure AI Agent service, conversations happen in "threads"
        # A thread maintains the context and history of a conversation
        # Here we create a new empty thread to start a fresh conversation
        thread = project_client.agents.create_thread()
        print(f"📝 Created new search thread, ID: {thread.id}")
        return thread
    except Exception as e:
        print(f"❌ Error creating search thread: {e}")
        return None

def ask_search_question(thread_id, agent_id, user_question):
    try:
        # First, we add the user's question as a message to the thread
        # This is like typing a message in a chat interface
        message = project_client.agents.create_message(
            thread_id=thread_id,
            role="user",  # Specifies this message is from the user
            content=user_question
        )
        print(f"🔎 Searching: '{user_question}'")

        # Next, we create and process a "run" - this is where the magic happens!
        # The agent will:
        # 1. Read the user's question
        # 2. Use the FileSearchTool to search our health documents
        # 3. Generate a helpful response based on the search results
        run = project_client.agents.create_and_process_run(
            thread_id=thread_id,
            assistant_id=agent_id
        )
        print(f"🤖 Run finished with status: {run.status}")
        if run.last_error:
            print(f"Error details: {run.last_error}")
        return run
    except Exception as e:
        print(f"❌ Error searching question: {e}")
        return None

# Now let's test our search functionality!
# First check if we have our health agent available
if health_agent:
    # Create a new conversation thread
    search_thread = create_search_thread(health_agent)

    if search_thread:
        # Define some test questions that demonstrate different types of health queries
        # The agent will search our uploaded health documents to answer these
        queries = [
            "Could you suggest a gluten-free lunch recipe?",
            "Show me some heart-healthy meal ideas.",
            "What guidelines do you have for someone with diabetes?"
        ]

        # Process each query one at a time
        # The agent will maintain conversation context between questions
        for q in queries:
            ask_search_question(search_thread.id, health_agent.id, q)

## 6. 결과 및 인용 보기 📄
대화 스레드를 읽고 에이전트가 어떻게 응답했는지, 올바른 파일을 인용했는지 확인합니다.

In [6]:
def display_thread_messages(thread_id):
    try:
        # Retrieve all messages in this conversation thread using the Azure AI Agent SDK
        # Messages contain the back-and-forth between user and AI agent
        messages = project_client.agents.list_messages(thread_id=thread_id)

        # Display the conversation history in reverse chronological order (newest first)
        print("\n🗣️ Conversation so far:")
        for m in reversed(messages.data):
            # Each message may have multiple content pieces
            # We're interested in the text content (vs other types like images)
            if m.content:
                last_content = m.content[-1]
                if hasattr(last_content, "text"):
                    # Print who said what (ASSISTANT or USER) along with their message
                    print(f"{m.role.upper()}: {last_content.text.value}\n")

        # The agent can cite specific passages from the uploaded documents
        # Let's check if it referenced any files in its responses
        print("\n📎 Checking for citations...")
        for c in messages.file_citation_annotations:
            # Each citation includes the quoted text and which file it came from
            # This helps users verify the agent's sources
            print(f"- Citation snippet: '{c.text}' from file ID: {c.file_citation['file_id']}")

    except Exception as e:
        # Gracefully handle any errors that might occur when displaying messages
        print(f"❌ Error displaying messages: {e}")

# Display the conversation history for our search thread
if search_thread:
    display_thread_messages(search_thread.id)

## 7. 정리 및 모범 사례 🧹
선택적으로 벡터 스토어, 업로드된 파일 및 에이전트를 제거합니다. 프로덕션 환경에서는 더 오래 보관할 수도 있습니다. 그 동안 몇 가지 팁을 알려드립니다:

1. **리소스 관리**
   - 파일을 카테고리별로 그룹화하고, 오래되거나 관련 없는 파일을 정기적으로 정리하세요.
   - 테스트 에이전트나 벡터 스토어는 작업이 끝나면 정리하세요.

2. **검색 쿼리**
   - 정확한 쿼리 또는 여러 부분으로 구성된 쿼리를 제공하세요.
   - 동의어 또는 대체 키워드를 고려하세요 (예를 들어 "gluten-free" vs "celiac").
   
3. **건강 정보**
   - 항상 의료 전문가가 아님을 명시하세요.
   - 사용자가 구체적인 진단을 위해 의사의 진찰을 받도록 권장하세요.

4. **성**
   - 벡터 스토어 크기의 변화에 주의하세요.
   - `azure-ai-evaluation`으로 검색 정확도를 평가하세요!


In [ ]:
def cleanup_all():
    try:
        # Check if we have a vector store and delete it
        # Vector stores are where we store the embeddings (numerical representations) 
        # of our documents for semantic search
        if 'vector_store' in globals() and vector_store:
            project_client.agents.delete_vector_store(vector_store.id)
            print("🗑️ Deleted vector store.")

        # Remove any files we uploaded to Azure AI Search
        # These were the documents our agent used as its knowledge base
        if 'file_ids' in globals() and file_ids:
            for fid in file_ids:
                project_client.agents.delete_file(fid)
            print("🗑️ Deleted uploaded files from the service.")

        # Delete the AI agent we created
        # This frees up resources since we're done with our demo
        if 'health_agent' in globals() and health_agent:
            project_client.agents.delete_agent(health_agent.id)
            print("🗑️ Deleted health resource agent.")

        # Clean up any local files we created during the demo
        # This keeps our workspace tidy
        if 'sample_files' in globals() and sample_files:
            for sf in sample_files:
                if os.path.exists(sf):
                    os.remove(sf)
            print("🗑️ Deleted local sample files.")

    except Exception as e:
        # If anything goes wrong during cleanup, we'll see what happened
        print(f"❌ Error during cleanup: {e}")

# Run our cleanup function to remove all resources we created
# This is good practice in a tutorial/demo environment
cleanup_all()

# 축하합니다! 🎉
**건강 리소스 검색 에이전트**를 만들었습니다:
1. **벡터 스토어**를 사용하여 샘플 레시피 및 가이드라인을 저장합니다.
2. 쿼리에 답하기 위해 **검색**합니다.
3. 사용자에게 전문가와 상담할 것을 상기시키는 **면책 조항**을 제공합니다.

기업 문서, 제품 설명서 또는 맞춤형 건강 리소스에 이 접근 방식을 자유롭게 적용하세요.

행복한 검색되세요! 🎉

#### [4-bing_grounding.ipynb](4-bing_grounding.ipynb)로 이동합니다.